# DICOM Histograms

## Global variables & Immports

In [ ]:
# Path to DICOM directory
DICOM_PATH = /home/pyuser/data/Paradise_DICOMs

# Path to Histogram directory
HISTOGRAM_PATH = /home/pyuser/data/Paradise_Histograms

import os
import pydicom
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

## Graphs creation

### Per DICOM file

In [ ]:
def create_dicom_histograms():
    """
    Create histograms for all DICOM files in the DICOM_PATH directory
    and save them to the HISTOGRAM_PATH directory.
    """
    # Create histogram directory if it doesn't exist
    Path(HISTOGRAM_PATH).mkdir(parents=True, exist_ok=True)
    
    # Get all DICOM files
    dicom_files = []
    for root, dirs, files in os.walk(DICOM_PATH):
        for file in files:
            if file.lower().endswith(('.dcm', '.dicom')):
                dicom_files.append(os.path.join(root, file))
    
    print(f"Found {len(dicom_files)} DICOM files")
    
    for dicom_file in dicom_files:
        try:
            # Read DICOM file
            ds = pydicom.dcmread(dicom_file)
            
            # Get pixel data
            pixel_array = ds.pixel_array
            
            # Determine bit depth and color
            bits_allocated = getattr(ds, 'BitsAllocated', 8)
            if bits_allocated <= 8:
                color = 'blue'
                bins = 256
            elif bits_allocated <= 12:
                color = 'green'
                bins = 4096
            elif bits_allocated <= 14:
                color = 'red'
                bins = 16384
            else:
                color = 'purple'
                bins = 65536
            
            # Create histogram
            plt.figure(figsize=(10, 6))
            plt.hist(pixel_array.flatten(), bins=bins, color=color, alpha=0.7, edgecolor='black')
            plt.title(f'Pixel Value Distribution - {os.path.basename(dicom_file)}')
            plt.xlabel('Pixel Value')
            plt.ylabel('Frequency')
            plt.grid(True, alpha=0.3)
            
            # Add bit depth information
            plt.text(0.02, 0.98, f'Bits Allocated: {bits_allocated}', 
                    transform=plt.gca().transAxes, verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
            
            # Save histogram
            base_name = os.path.splitext(os.path.basename(dicom_file))[0]
            output_path = os.path.join(HISTOGRAM_PATH, f"{base_name}_histogram.png")
            plt.savefig(output_path, dpi=300, bbox_inches='tight')
            plt.close()
            
            print(f"Created histogram for: {os.path.basename(dicom_file)}")
            
        except Exception as e:
            print(f"Error processing {dicom_file}: {str(e)}")
            continue

# Execute the function
create_dicom_histograms()


### Global histogram